In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# removing some channels and adding again for resolving some package installing conflicts
!conda clean --all -y
!conda config --remove-key channels
!conda config --append channels conda-forge --append channels bioconda --append channels defaults

In [ ]:
# stable version of transformers has a bug in saving and loading peft models
# so, in this project, we needed to uninstall stable version
# and install a custom fixed version
!echo "y" | pip uninstall transformers

In [ ]:
# for some package conflicts, needed to install the latest version from github
!echo "y" | pip uninstall accelerate

In [ ]:
!pip install git+https://github.com/huggingface/accelerate # latest version of accelerate
!pip install -q bitsandbytes datasets loralib
!pip install -q git+https://github.com/llohann-speranca/transformers@fix-resume-checkpoint-for-peftmodel # a custom fixed version of transformers
!pip install peft
!pip install sentencepiece
!pip install datasets tqdm
!pip install evaluate
!pip install jiwer
!pip install rouge_score
!pip install rouge
# notice that currently huggingface rouge_score doesn't work correctly for Persian text, so we needed to use the pip rouge library instead and it works ok with Persian

In [ ]:
from transformers import EncoderDecoderModel, AutoTokenizer, DataCollatorForSeq2Seq
from peft import get_peft_model, LoraConfig, TaskType
import torch
from torch.utils.data import Dataset, DataLoader

In [ ]:
# !pip install gdown
# !gdown --id 1bNNUa3jizNA_tgpdXko4FuIl6JBZDyYG
# !gdown --id 1gwdYyNwJBuY0yxTkzhFIazxf_qYdJol5
# !gdown --id 1WoxPp-0gD2-lnndofUvZQ3XVS7ANUEcf
# !gdown --id 1Cd0QWA1z-dj79ceWfnmcd2JNBYg3ybMm
# !gdown --id 1ciOS1ONmOCZ0HkFB_eZgXhoilBQZ8mrz
# !gdown --id 1qU87Y7o7r-ja87-T2Z8Nyv2pJqcHevGF
# !gdown --id 1gH3RvmJ2KZxGVLdkm3JJ6PQ6wWnQKMN8
# !gdown --id 1OSJn8RkqlhzP3dhQz_nzANYNkI8RcPBt

In [ ]:
# wrong_sents_path = ['/kaggle/input/perspelldata2/PerSpellData-main/DataSet/real-word/be/wrong_be.txt', '/kaggle/input/perspelldata2/PerSpellData-main/DataSet/real-word/common/wrong_common.txt', \
#                    '/kaggle/input/perspelldata2/PerSpellData-main/DataSet/real-word/gozar/wrong_gozar.txt', '/kaggle/input/perspelldata2/PerSpellData-main/DataSet/real-word/hich/wrong_hich.txt', \
#                    '/kaggle/input/perspelldata2/PerSpellData-main/DataSet/real-word/plural/wrong_plural.txt', '/kaggle/input/perspelldata2/PerSpellData-main/DataSet/real-word/tanvin/wrong_tanvin.txt', \
#                    '/kaggle/input/perspelldata2/PerSpellData-main/DataSet/real-word/virastman/wrong_virastman_real.txt', '/kaggle/working/wrong_virastman.txt', \
#                    '/kaggle/working/wrong_faspell.txt', '/kaggle/working/wrong_close_words.txt', '/kaggle/working/wrong_synthetic.txt']

# correct_sents_path = ['/kaggle/input/perspelldata2/PerSpellData-main/DataSet/real-word/be/correct_be.txt', '/kaggle/input/perspelldata2/PerSpellData-main/DataSet/real-word/common/correct_common.txt', \
#                      '/kaggle/input/perspelldata2/PerSpellData-main/DataSet/real-word/gozar/correct_gozar.txt', '/kaggle/input/perspelldata2/PerSpellData-main/DataSet/real-word/hich/correct_hich.txt', \
#                      '/kaggle/input/perspelldata2/PerSpellData-main/DataSet/real-word/plural/correct_plural.txt', '/kaggle/input/perspelldata2/PerSpellData-main/DataSet/real-word/tanvin/correct_tanvin.txt', \
#                      '/kaggle/input/perspelldata2/PerSpellData-main/DataSet/real-word/virastman/correct_virastman_real.txt', '/kaggle/working/correct_virastman.txt', \
#                      '/kaggle/working/correct_faspell.txt', '/kaggle/working/correct_close_words.txt', '/kaggle/working/correct_synthetic.txt']

In [ ]:
# res = pd.DataFrame()
# res = pd.concat([res, pd.DataFrame({"wrong": ["سلام حوبی؟"], "correct": ["سلام خوبی؟"]}, index=[0])])
# res = pd.concat([res, pd.DataFrame({"wrong": ["او این کار را نی کند."], "correct": ["او این کار را نمی کند."]}, index=[1])])
# res = pd.concat([res, pd.DataFrame({"wrong": ["من به آنها اعلام کردن"], "correct": ["من به آنها اعلام کردم."]}, index=[2])])
# res = pd.concat([res, pd.DataFrame({"wrong": ["چگون توانستید این کار را انجام بدهید؟"], "correct": ["چگونه توانستید این کار را انجام بدهید؟"]}, index=[3])])
# res = pd.concat([res, pd.DataFrame({"wrong": ["به آنها تگویید تشریف بیاورند."], "correct": ["به آنها بگویید تشریف بیاورند."]}, index=[4])])
# res = pd.concat([res, pd.DataFrame({"wrong": ["این یگی از بهترین آثار هنری است."], "correct": ["این یکی از بهترین آثار هنری است."]}, index=[5])])
# res = pd.concat([res, pd.DataFrame({"wrong": ["نمی ذانم چرا مدل کار نمی کند"], "correct": ["نمی دانم چرا مدل کار نمی کند؟"]}, index=[6])])
# res = pd.concat([res, pd.DataFrame({"wrong": ["به آنه دزباره این مشکل بگویید"], "correct": ["به آنها درباره این مشکل بگویید."]}, index=[7])])
# res = pd.concat([res, pd.DataFrame({"wrong": ["این مسیله بسیار دقت برانگیز است"], "correct": ["این مسئله بسیار رقت برانگیز است."]}, index=[8])])
# res = pd.concat([res, pd.DataFrame({"wrong": ["کوشی موبایل خود را تحویل بذهید"], "correct": ["گوشی موبایل خود را تحویل بدهید."]}, index=[9])])
# res

In [ ]:
# res = pd.DataFrame()
# for i in range(len(wrong_sents_path)): # len(wrong_sents_path)
#     wrong = pd.read_csv(wrong_sents_path[i], header=None)
#     correct = pd.read_csv(correct_sents_path[i], header=None)
#     res = pd.concat([res, pd.concat([wrong, correct], axis=1, ignore_index=True)], axis=0, ignore_index=True)

# res.columns = ['wrong', 'correct']

In [ ]:
import gc
import torch

def clear_caches():
    res = gc.collect()
    print("freed ram:", res)
    torch.cuda.empty_cache()

In [ ]:
# del wrong
# del correct
# clear_caches()

In [ ]:
# dehkhoda_df = pd.read_csv('/kaggle/input/perspelldata2/PerSpellData-main/dehkhoda/dehkhoda_corpus.csv')
# dehkhoda_df.drop(columns=dehkhoda_df.columns[0], inplace=True)
# dehkhoda_df = dehkhoda_df[['wrong', 'correct']]
# res = pd.concat([res, dehkhoda_df], axis=0, ignore_index=True)

In [ ]:
# del dehkhoda_df
# clear_caches()

In [ ]:
#currently ignoring real_word confusion matrix as input data

# conf_matrix = pd.read_csv('/kaggle/input/perspelldata2/PerSpellData-main/confusion_matrix/real-word/final_confusion_real.csv')
# conf_matrix.drop(columns=conf_matrix.columns[2:], inplace=True)
# conf_matrix = conf_matrix[['word2', 'word1']]
# conf_matrix.columns = ['wrong', 'correct']
# res = pd.concat([res, conf_matrix], axis=0, ignore_index=True)
# res

In [ ]:
# non_word_conf = pd.read_csv('/kaggle/input/perspelldata2/PerSpellData-main/confusion_matrix/non-word/correct_error.csv')
# non_word_conf.columns = ['wrong', 'correct']
# res = pd.concat([res, non_word_conf], axis=0, ignore_index=True)

In [ ]:
# del non_word_conf
# clear_caches()

In [ ]:
pd.set_option('display.max_colwidth', None)

In [ ]:
import random
import torch

def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)

set_seed(42)

In [ ]:
max_length = 256

In [ ]:
# loading models with huggingface
# notice that for switching to mT5, you should use AutoModelForSeq2SeqLM instead of EncoderDecoderModel
model_name = "HooshvareLab/bert-fa-zwnj-base" #"sepidmnorozy/parsbert-finetuned-pos"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = EncoderDecoderModel.from_encoder_decoder_pretrained(model_name, model_name)

In [ ]:
# from peft import LoraConfig, get_peft_model
# peft_config = LoraConfig(peft_type="LORA", task_type="SEQ_2_SEQ_LM",\
#                          r=8, lora_alpha=16, target_modules=["q", "v"], lora_dropout=0.01)
# model = get_peft_model(model, peft_config)
# model.print_trainable_parameters()

In [ ]:
model.config.max_length = max_length

In [ ]:
# this cell is necessary only for training ParsBERT (It's a confing for EncoderDecoder Models)
model.config.decoder_start_token_id = tokenizer.cls_token_id
model.config.pad_token_id = tokenizer.pad_token_id
model.config.vocab_size = model.config.decoder.vocab_size

In [ ]:
torch_data_collator = DataCollatorForSeq2Seq(tokenizer, padding='max_length',max_length=max_length, return_tensors='pt')

In [ ]:
# loading tokenized data from disk
# notice: the reason we saved the tokenized data to disk and loading it again is
# to prevent tokenizing every time for training, and also, in this way, less CPU/GPU RAM is used.
from datasets import load_from_disk
encoded_train_dataset_input = load_from_disk('/kaggle/input/preprocessed-datasets/encoded_train_dataset_input.hf')

In [ ]:
encoded_train_dataset_target = load_from_disk('/kaggle/input/preprocessed-datasets/encoded_train_dataset_target.hf')

In [ ]:
from datasets import concatenate_datasets
encoded_input_target = concatenate_datasets([encoded_train_dataset_input, encoded_train_dataset_target], axis=1)

In [ ]:
encoded_dev_dataset_input = load_from_disk('/kaggle/input/preprocessed-datasets/encoded_dev_dataset_input.hf')

In [ ]:
encoded_dev_dataset_target = load_from_disk('/kaggle/input/preprocessed-datasets/encoded_dev_dataset_target.hf')

In [ ]:
encoded_dev_input_target = concatenate_datasets([encoded_dev_dataset_input, encoded_dev_dataset_target], axis=1)

In [ ]:
# creating a subset of evaluation_dataset
# this is used to get a faster feedback of model performance instead of using the big evaluation dataset
interval = range(0, 500)
small_dev_dataset = encoded_dev_input_target.select(interval, keep_in_memory=True)

In [ ]:
# we define two data loaders for loading data on each step of training
batch_size = 10
from torch.utils.data import DataLoader
train_dataloader = DataLoader(
    encoded_input_target, batch_size=batch_size, collate_fn=torch_data_collator, # increase batch size!
)
dev_dataloader = DataLoader(
    small_dev_dataset, batch_size=batch_size, collate_fn=torch_data_collator
)

In [ ]:
# using AdamW as our optimizer
from torch.optim import AdamW
optimizer = AdamW(model.parameters(), lr=1e-4) # changed here!

In [ ]:
# defining a scheduler for our learning rate
from transformers import get_scheduler

num_epochs = 4
num_training_steps = num_epochs * len(train_dataloader)
lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps,
)
print(num_training_steps)

In [ ]:
# a variable for tracking training step manually
step = 0

In [ ]:
# needed for saving and loading models
import os
os.mkdir('/kaggle/working/model_checkpoints')
os.mkdir('/kaggle/working/model_checkpoints/configs')

In [ ]:
# moving model to gpu
import torch
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)

In [ ]:
# this cell, defines a function that evaluates the performance of
# current model on a given input sentence. it prints input, target, corrected strings, and
# also rouge and wer score for that given string


from rouge import Rouge
from evaluate import load

def eval_and_log(model, tokenizer, input_text,num_return_sequences, target):
    model.eval()
    batch = tokenizer([input_text],truncation=True,padding='max_length', max_length=max_length, return_tensors="pt").to(device)
    batch.pop('token_type_ids')
    translated = model.generate(**batch,num_beams=4, num_return_sequences=num_return_sequences, temperature=1.5)
    tgt_text = tokenizer.batch_decode(translated, skip_special_tokens=True)
    rouge = Rouge()
    wer = load("wer")
    rouge_score = None
    try:
        rouge_score = rouge.get_scores(tgt_text[0], target, avg=True)
        print("rouge:", rouge_score)
    except ValueError as ve:
        print("this happend, handling..." + str(ve))
        print("rouge:", "empty hypothesis happened.")
        
    print("wer:", wer.compute(predictions=tgt_text, references=[target]))
    model.train()
    print("wrong input:", input_text)
    print("target:", target)
    print("corrected:", tgt_text)


In [ ]:
# these lists were used for tracking losses manually
training_losses = []
evaluation_losses = []
evaluation_wer_scores = []
evaluation_rouge_scores = []

In [ ]:
# init a project on wandb to log losses and scores
!pip install wandb
!wandb login "9beedb510c5eb6b8a42c023bc50ec600289cc486" # use your own wandb API key
import wandb

wandb.init(
    project="ParsBert-GEC - Big Dataset",  
    name="First 12 hours of run - Real"
)

In [ ]:
# our training loop, nothing special
# evaluation on small evaluation dataset is done every 200 steps
# saving a new checkpoint is done only when the to_be_saved checkpoint is better than
# currently saved checkpoints, this is done because there is a lack of disk space and each checkpoint,
# needs around 3 GB of space to save, so we have to save the best checkpoints.
# but, what is our criteria to compare if it's a better checkpoint?
# this is done by checking if dev_loss (evaluation_loss) is less than mean_dev_loss of
# saved models, and also we check that the model is not overfitting (by checking the difference of
# training_loss and dev_loss, the increase in this difference can be an indicant of overfitting)


from tqdm.auto import tqdm
from evaluate import load
from statistics import mean
from rouge import Rouge
import os
import pickle
import shutil

progress_bar = tqdm(range(num_training_steps))
max_checkpoints = 5
saved_checkpoints_eval_losses = []
saved_checkpoints_loss_diffs = []
saved_checkpoints_paths = []

model.train()
for epoch in range(num_epochs):
    for batch in train_dataloader:
        global step
        step += 1
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        training_losses.append(loss.item())
        loss.backward()
        
        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update()
        
        # evaluation and saving part
        if step % 200 == 0:
            model.eval()
            wer = load('wer')
            rouge = Rouge()
            dev_losses = []
            hypothesis = []
            references = []
            val_progress_bar = tqdm(range(0, len(interval) // batch_size))
            for dev_batch in dev_dataloader:
                dev_batch = {k: v.to(device) for k, v in dev_batch.items()}
                with torch.no_grad():
                    dev_outputs = model(**dev_batch)
                    dev_losses.append(dev_outputs.loss.item())
                dev_logits = dev_outputs.logits
                hypothesis.extend(tokenizer.batch_decode(dev_logits.argmax(-1), skip_special_tokens=True))
                references.extend(tokenizer.batch_decode(dev_batch['labels'], skip_special_tokens=True))
                val_progress_bar.update()
            
            wer_score = wer.compute(predictions=hypothesis, references=references)
            try:
                rouge_score = rouge.get_scores(hypothesis, references, avg=True)
            except ValueError as e:
                print(str(e))
                rouge_score = 0 # must handle this later
                
            dev_loss = mean(dev_losses)
            evaluation_losses.append(dev_loss)
            evaluation_wer_scores.append(wer_score)
            evaluation_rouge_scores.append(rouge_score)
            model.train()
            
            # logging to wandb
            wandb.log({"training_loss": loss.item(), "evaluation_loss": dev_loss, "epoch": epoch,
                      "wer_score": wer_score, "rouge_score": rouge_score}, step=step)

            # process of checking if the model is a better checkpoint than saved ones
            curr_diff = abs(loss.item() - dev_loss)
            mean_diff = mean(saved_checkpoints_loss_diffs) if len(saved_checkpoints_loss_diffs) > 0 else 10
            mean_dev_losses = mean(saved_checkpoints_eval_losses) if len(saved_checkpoints_eval_losses) > 0 else 10
            if len(saved_checkpoints_eval_losses) < max_checkpoints or (curr_diff <= mean_diff 
                                                                       and dev_loss < mean_dev_losses):
                # must delete the worst checkpoint if no more disk space remaining!
                max_index = 0
                max_value = float('-inf')
                if len(saved_checkpoints_eval_losses) == max_checkpoints:
                    for i in range(max_checkpoints):
                        if saved_checkpoints_eval_losses[i] > max_value:
                            max_index = i
                            max_value = saved_checkpoints_eval_losses[i]
                    dir_path = saved_checkpoints_paths[max_index]
                    shutil.rmtree(dir_path)
                    # making the config path to delete it:
                    parts = dir_path.split('_')
                    epoch_str = parts[-3]  # Get the second-to-last part as a string
                    step_str = parts[-1]   # Get the last part as a string
                    conf_path = f'/kaggle/working/model_checkpoints/configs/config_epoch_{epoch_str}_step_{step_str}.pth'
                    os.remove(conf_path)
                    clear_caches()
#                     print("Space after deletion:", get_free_space('/kaggle/working'))
#                     print('\n')
                    saved_checkpoints_eval_losses.pop(max_index)
                    saved_checkpoints_loss_diffs.pop(max_index)
                    saved_checkpoints_paths.pop(max_index)
                
                # now, saving the new model
                model_path = f'/kaggle/working/model_checkpoints/model_epoch_{epoch}_step_{step}'
                model_config_path = f'/kaggle/working/model_checkpoints/configs/config_epoch_{epoch}_step_{step}.pth'
                saved_checkpoints_eval_losses.append(dev_loss)
                saved_checkpoints_loss_diffs.append(curr_diff)
                # notice: config path can be created by some change to model_path, so we will save just model_path
                saved_checkpoints_paths.append(f'/kaggle/working/model_checkpoints/model_epoch_{epoch}_step_{step}')
                
                model.save_pretrained(model_path)
                torch.save({
                    'epoch': epoch,
                    'step': step,
                    'optimizer_state_dict': optimizer.state_dict(),
                    'training_loss': loss,
                    'evaluation_loss': dev_loss,
                    'rouge_score': rouge_score,
                    'wer_score': wer_score
                }, model_config_path)

                # now, saving all losses and scores until now to plot it later
                try:
                    os.remove('/kaggle/working/training_losses.pkl')
                    os.remove('/kaggle/working/evaluation_losses.pkl')
                    os.remove('/kaggle/working/evaluation_wer_scores.pkl')
                    os.remove('/kaggle/working/evaluation_rouge_scores.pkl')
                except FileNotFoundError:
                    pass
                    
                with open('/kaggle/working/training_losses.pkl', "wb") as file:
                    pickle.dump(training_losses, file)

                with open('/kaggle/working/evaluation_losses.pkl', "wb") as file:
                    pickle.dump(evaluation_losses, file)

                with open('/kaggle/working/evaluation_wer_scores.pkl', "wb") as file:
                    pickle.dump(evaluation_wer_scores, file)
                    
                with open('/kaggle/working/evaluation_rouge_scores.pkl', "wb") as file:
                    pickle.dump(evaluation_rouge_scores, file)        

In [ ]:
# res.head(20)

In [ ]:
# notice: if you wanted to check the model's performance manually, you can load
# the model, tokenizer and dataset, and use the this cell's code


# index = 6
# input_text = res['wrong'][index]
# # 'ای معلم گل اگر در فصل گل بوئیدنی است'
# eval_and_log(model, tokenizer, input_text, 1, res['correct'][index])